In [1]:
from pathlib import Path
import anndata
import torch
names = ['BI','KLA','KLB','LA','MA','TIR','TIS']
adatas = {}
for name in names:
    adatas[name] = anndata.read_h5ad('/data/chensq/data/sccancer/datasets_1018/'+name+'.h5ad')

In [2]:
#使用GPU
torch.cuda.is_available()
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
torch.device("cuda:0")

Using cuda:0 device


device(type='cuda', index=0)

In [3]:
from scCrab import *
pred, prob = scCrab(adatas=adatas,train_name="BI",reference_name="TIR",test_name="MA",device="cuda:0",
       main_obs='major_ct', detail_obs='sub_ct', save_path="/home/chensq/data/sccancer/data/hhy/inter/")

Using cuda:0 device


/home/chensq/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chensq/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/chensq/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/chensq/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/chensq/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names

Using cuda:0 device


/home/chensq/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


resetting params
Epoch 1 	 Training Loss: 0.3598029140823631 	 Validation Loss: 0.23740855147761683 	 Time: 6.202981948852539
Epoch 2 	 Training Loss: 0.10141633883098601 	 Validation Loss: 0.012342614992972343 	 Time: 4.347141981124878
Epoch 3 	 Training Loss: 0.00839257109672412 	 Validation Loss: 0.006566241683979188 	 Time: 5.952158451080322
Epoch 4 	 Training Loss: 0.005982150423831196 	 Validation Loss: 0.0053178682283032686 	 Time: 6.184927225112915
Epoch 5 	 Training Loss: 0.004715133467451319 	 Validation Loss: 0.005152357051209096 	 Time: 6.033747673034668
Epoch 6 	 Training Loss: 0.004072160348333719 	 Validation Loss: 0.0041936912457458675 	 Time: 5.838235378265381
Epoch 7 	 Training Loss: 0.003298110557220606 	 Validation Loss: 0.003769029387190277 	 Time: 5.971690654754639
Epoch 8 	 Training Loss: 0.004030831742584856 	 Validation Loss: 0.0037507483427743275 	 Time: 5.957723379135132
Model Converge
2


Less than 80% of signature genes are available in data set. A temporary signature is stored where non-overlapping genes are removed. It is proceeded with the temporary signature.
Less than 80% of signature genes are available in data set. A temporary signature is stored where non-overlapping genes are removed. It is proceeded with the temporary signature.


/home/chensq/.local/lib/python3.10/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


converged at iteration step: 6 with 0.0002 < 0.001


In [12]:
AUPRC, bas, kappa = evaluate_metrics(adatas, test_name="MA", main_obs="major_ct", pred=pred, prob=prob)

In [13]:
print(AUPRC, bas, kappa)

0.7369736412506425 0.6308131830000947 0.32940263843375384


In [1]:
from pathlib import Path
import anndata
import torch
paths = [
    Path("/data/chensq/data/sccancer/data/ikarus/bischoff22_carcinoid/adata.h5ad"),
    Path("/data/chensq/data/sccancer/data/ikarus/kildisiute21_neuroblastoma_10x/adata.h5ad"),
    Path("/data/chensq/data/sccancer/data/ikarus/kildisiute21_neuroblastoma_celseq2/adata.h5ad"),
    Path("/data/chensq/data/sccancer/data/ikarus/lambrechts18_lung/adata.h5ad"),
    Path("/data/chensq/data/sccancer/data/ikarus/ma21_liver/adata.h5ad"),
    Path("/data/chensq/data/sccancer/data/ikarus/tirier21_myeloma/adata.h5ad"),
    Path("/data/chensq/data/sccancer/data/ikarus/tirosh17_headneck/adata.h5ad"),
]
names = [
    "bischoff",
    "kildisiute_10x",
    "kildisiute_celseq2",
    "lambrechts",
    "ma",
    "tirier",
    "tirosh",
]
adatas = {}
for path, name in zip(paths, names):
    adatas[name] = anndata.read_h5ad(path)

In [5]:
#modify datasets
obs_list = ["tier_2", "major", "major", "major", "tier_1", "tier_1", "major"]
new_names = ['BI','KLA','KLB','LA','MA','TIR','TIS']
for (name, obs_name, new_name) in zip(names, obs_list, new_names):
    adata = adatas[name]
    adata.obs['major_ct'] = adata.obs['tier_0']
    adata.obs['sub_ct'] = adata.obs[obs_name]
    adata.write_h5ad('/data/chensq/data/sccancer/datasets_1018/'+new_name+'.h5ad')